# 🔍Sanity Check

This notebook verifies that the Dev Container (Python 3.13 + CUDA 13) can access the GPU through PyTorch and perform GPU accelerated computations via RAPIDS.

### GPU, CUDA, RAPIDS Sanity Check

In [1]:
# must be BEFORE importing pandas
%load_ext cudf.pandas

/workspaces/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %code_wrap  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %mamba  %man  %matplotlib  %micromamba  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %subshell  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %uv  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%code_wrap  %%cudf.pandas.line_profile  %%

In [3]:
import IPython
import pandas as pd
import cudf
import cudf.pandas
import cupy
import torch
import platform

print("Python:                  ", platform.python_version())
print("IPython:                 ", IPython.__version__)
print("Magics present:          ","cudf.pandas.profile" in get_ipython().magics_manager.magics['cell'])
print()

if torch.cuda.is_available():
    print("GPU:                     ", torch.cuda.get_device_name(0))
    print("CUDA runtime version:    ", torch.version.cuda)
    print("cuDF:                    ", cudf.__version__)
    print("cudf.pandas installed:   ", hasattr(cudf.pandas, "install"))
    print("cupy config:             ", cupy.show_config())

    print()

    print("PyTorch:                 ", torch.__version__)
    print("CUDA available (torch):  ", torch.cuda.is_available())

Python:                   3.13.8
IPython:                  9.7.0
Magics present:           True

GPU:                      NVIDIA GeForce RTX 4090 Laptop GPU
CUDA runtime version:     13.0
cuDF:                     25.10.00
cudf.pandas installed:    True
OS                           : Linux-6.14.0-35-generic-x86_64-with-glibc2.39
Python Version               : 3.13.8
CuPy Version                 : 13.6.0
CuPy Platform                : NVIDIA CUDA
NumPy Version                : 2.2.6
SciPy Version                : 1.16.3
Cython Build Version         : 3.0.12
Cython Runtime Version       : None
CUDA Root                    : /usr/local/cuda
nvcc PATH                    : /usr/local/cuda/bin/nvcc
CUDA Build Version           : 13000
CUDA Driver Version          : 13000
CUDA Runtime Version         : 13000 (linked to CuPy) / 13000 (locally installed)
CUDA Extra Include Dirs      : ['/workspaces/.venv/lib/python3.13/site-packages/nvidia/cu13/include']
cuBLAS Version               : (availab

### cuDF

In [4]:
%%cudf.pandas.profile

# cuDF accelerated data frame
df = cudf.DataFrame({"a": range(1_000_000)})
print("sum (cuDF):", int(df["a"].sum()))

sum (cuDF): 499999500000


                                                                                              
                                  Total time elapsed: 0.154 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

In [5]:
%%cudf.pandas.profile

df = pd.DataFrame({"a": range(1_000_000)})
print("sum (pandas-accelerated):", df["a"].sum())

sum (pandas-accelerated): 499999500000


                                                                                                           
                                         Total time elapsed: 0.121 seconds                                 
                                       3 GPU function calls in 0.003 seconds                               
                                       0 CPU function calls in 0.000 seconds                               
                                                                                                           
                                                       Stats                                               
                                                                                                           
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function              ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ DataFrame             │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.__getitem__ │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Series.sum            │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
└───────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

In [6]:
%%cudf.pandas.profile

# cuDF implicit via pandas-accelerated
print("pandas version:", pd.__version__)
df = pd.DataFrame({"x": range(10_000_000)})
_ = df.groupby(pd.cut(df["x"], 100)).x.mean()

pandas version: 2.3.3


                                                                                                           
                                         Total time elapsed: 0.385 seconds                                 
                                       6 GPU function calls in 0.073 seconds                               
                                       0 CPU function calls in 0.000 seconds                               
                                                                                                           
                                                       Stats                                               
                                                                                                           
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function              ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ DataFrame             │ 1          │ 0.002       │ 0.002       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.__getitem__ │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ cut                   │ 1          │ 0.034       │ 0.034       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.groupby     │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ GroupBy.__getattr__   │ 1          │ 0.002       │ 0.002       │ 0          │ 0.000       │ 0.000       │
│ GroupBy.mean          │ 1          │ 0.034       │ 0.034       │ 0          │ 0.000       │ 0.000       │
└───────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

### cuML

In [7]:
import time
import cupy as cp
import statistics as stats

from cuml.model_selection import train_test_split
from cuml.linear_model import LogisticRegression

#### Benchmark Helper

In [8]:
def benchmark_gpu(fn, *args, label=None, **kwargs):
    """
    Benchmark a GPU function call using both CPU wall time and CUDA events.

    Args:
        fn: Callable to benchmark (e.g., clf.fit)
        *args, **kwargs: Arguments to pass to fn
        label: Optional string for print labeling

    Returns:
        result: return value from fn
        metrics: dict with 'cpu_ms', 'gpu_ms', 'overhead_ms'
    """
    start_gpu = cp.cuda.Event()
    end_gpu = cp.cuda.Event()
    cp.cuda.Device().synchronize()

    start_cpu = time.perf_counter()
    start_gpu.record()

    result = fn(*args, **kwargs)

    end_gpu.record()
    end_gpu.synchronize()
    end_cpu = time.perf_counter()

    cpu_ms = (end_cpu - start_cpu) * 1000
    gpu_ms = cp.cuda.get_elapsed_time(start_gpu, end_gpu)
    overhead_ms = cpu_ms - gpu_ms

    if label:
        print(f"\n[{label}]")
    print(f"CPU wall time : {cpu_ms:8.3f} ms")
    print(f"GPU kernel time: {gpu_ms:8.3f} ms")
    print(f"Overhead      : {overhead_ms:8.3f} ms")

    return result, {"cpu_ms": cpu_ms, "gpu_ms": gpu_ms, "overhead_ms": overhead_ms}

#### cuML Benchmark on GPU

In [9]:
# Data generation
N, D = 200_000, 20
cp.random.seed(42)

X = cp.random.rand(N, D, dtype=cp.float32)
y = (X.sum(axis=1) > 10).astype(cp.int32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Logistic Regression fit benchmark
clf = LogisticRegression(max_iter=100)
_, fit_metrics = benchmark_gpu(clf.fit, X_train, y_train, label="Fit")

# Logistic Regression predict benchmark
y_pred, pred_metrics = benchmark_gpu(clf.predict, X_test, label="Predict")
acc = float((y_pred == y_test).mean())
print(f"\nAccuracy: {acc:.4f}")

cpu_times, gpu_times, overhead_times = [], [], []

# Repeat multiple runs (skip first for warm-up)
for i in range(6):
    _, metrics = benchmark_gpu(clf.fit, X_train, y_train, label=f"Fit {i}")
    if i == 0:
        continue  # skip warm-up
    cpu_times.append(metrics["cpu_ms"])
    gpu_times.append(metrics["gpu_ms"])
    overhead_times.append(metrics["overhead_ms"])

def summarize(label, data):
    return (f"{label:>10}: "
            f"mean={stats.mean(data):8.3f} ms | "
            f"median={stats.median(data):8.3f} ms | "
            f"min={min(data):8.3f} | max={max(data):8.3f}")

print(f"\nResults over {len(cpu_times)} runs (excluding warm-up):")
print(summarize("CPU", cpu_times))
print(summarize("GPU", gpu_times))
print(summarize("Overhead", overhead_times))


[Fit]
CPU wall time :  681.352 ms
GPU kernel time:  681.346 ms
Overhead      :    0.006 ms

[Predict]
CPU wall time :   62.848 ms
GPU kernel time:   62.837 ms
Overhead      :    0.011 ms

Accuracy: 0.9994

[Fit 0]
CPU wall time :   15.746 ms
GPU kernel time:   15.729 ms
Overhead      :    0.017 ms

[Fit 1]
CPU wall time :   19.379 ms
GPU kernel time:   19.370 ms
Overhead      :    0.009 ms

[Fit 2]
CPU wall time :   15.318 ms
GPU kernel time:   15.309 ms
Overhead      :    0.009 ms

[Fit 3]
CPU wall time :   15.075 ms
GPU kernel time:   15.066 ms
Overhead      :    0.009 ms

[Fit 4]
CPU wall time :   14.847 ms
GPU kernel time:   14.833 ms
Overhead      :    0.014 ms

[Fit 5]
CPU wall time :   16.292 ms
GPU kernel time:   16.283 ms
Overhead      :    0.009 ms

Results over 5 runs (excluding warm-up):
       CPU: mean=  16.182 ms | median=  15.318 ms | min=  14.847 | max=  19.379
       GPU: mean=  16.172 ms | median=  15.309 ms | min=  14.833 | max=  19.370
  Overhead: mean=   0.010 ms

### Torch Sanity Check

In [10]:
# Run a small GPU tensor operation to confirm GPU compute works
if torch.cuda.is_available():
    x = torch.rand((10000, 10000), device="cuda")
    y = torch.rand((10000, 10000), device="cuda")
    z = torch.mm(x, y)
    print("✅ GPU matrix multiplication completed successfully.")
    del x, y, z
else:
    print("⚠️  Skipping GPU compute test (CUDA not available).")

✅ GPU matrix multiplication completed successfully.
